<a href="https://colab.research.google.com/github/msaif28/GenAI/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[sentensepiece] datasets sacrebleu rouge_score py7zr -q
!pip install sentencepiece

In [2]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install tensorflow

Requirement already up-to-date: accelerate in ./demo/notebookenv/lib/python3.8/site-packages (0.27.2)
Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay


2024-03-03 12:55:11.058811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-03 12:55:11.081231: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 12:55:11.545035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Building Own Translation system

# language translation system

# from the hugging face we are going to use opensource translation model

# dataset also will take from huggingface

Google - T5
Facebook - M2M100
Google cloud Translation API
Tune AI
Amazon comprehend
Helensinki

In [4]:
model="Helsinki-NLP/opus-mt-en-hi"

In [5]:
dataset=load_dataset("cfilt/iitb-english-hindi")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [7]:
dataset['train']

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [8]:
dataset['test']

Dataset({
    features: ['translation'],
    num_rows: 2507
})

In [9]:
dataset['validation']

Dataset({
    features: ['translation'],
    num_rows: 520
})

In [10]:
dataset['train'][2]

{'translation': {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}}

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

/home/saifm/demo/notebookenv/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
tokenizer("there is sunny who is teachine data science")

{'input_ids': [188, 23, 41670, 66, 23, 2144, 6131, 1195, 7373, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(['निचले पटल के लिए डिफोल्ट प्लग-इन खाका']))

{'input_ids': [[18464, 12422, 6, 39, 22433, 10076, 69, 4553, 7788, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/home/saifm/demo/notebookenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [14]:
dataset['train']['translation']

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'The default plugin layout for the top panel',
  'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'A list of plugins that are disabled by default',
  'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
 {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
 {'en': 'The duration of the highlight box when selecting accessible nodes',
  'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
 {'en': 'Highlight border color',
  'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
 {'en': 'The color and opacity of the highlight border.',
  'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
 {'en': 'Highlight fill color', 'hi': 

In [15]:
for ex in dataset['train']['translation']:
  print(ex['en'])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [16]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2507
    })
})

In [19]:
model

'Helsinki-NLP/opus-mt-en-hi'

In [20]:
#Here am loading the pretrained model form huggingface hub
model=TFAutoModelForSeq2SeqLM.from_pretrained(model)

2024-03-03 12:55:35.145583: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [21]:
batch_size=32
learning_rate=0.0001
weight_decay=0.01
num_train_epochs=1

In [22]:
#data collator form combining token and model

data_collator=DataCollatorForSeq2Seq(tokenizer,model=model, return_tensors="tf")

In [23]:
!nvidia-smi

Sun Mar  3 12:55:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:21:00.0 Off |                  Off |
| 30%   36C    P8              27W / 300W |     12MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [24]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],  # i am using test test data instead of train data large
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [25]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [26]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [27]:
#this is am doing fine tuning
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

78/78 [==============================] - 149s 2s/step - loss: 3.6388 - val_loss: 3.8662


**Model Inference**

In [28]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [34]:
input_text="i am learning coding today"

In [35]:
tokenized=tokenizer([input_text],return_tensors='np')

In [36]:
tokenized

{'input_ids': array([[5556,  489, 3729, 7308,  232,  137,  765,    0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [37]:
out=model.generate(**tokenized, max_length=128)

In [38]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

आज मैं सीख रहा हूँ
